<a href="https://colab.research.google.com/github/anshupandey/Computer-Vision/blob/master/Image%20Segmentation/Semantic_Segmentation_using_PASCAL_VOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Download Pascal VOC Dataset

In [ ]:
#Get PASCAL VOC dataset
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar --quiet

In [ ]:
!ls -l

In [ ]:
#unzip data
!tar -xf VOCtrainval_06-Nov-2007.tar

In [ ]:
!ls -l

In [ ]:
!ls -l images

In [ ]:
!ls -l masks

Images and Masks are unzipped

In [ ]:
!ls -l images | wc -l

#### Visualizing Data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import PIL

Read training images and build a dataframe with IDs

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
df.shape

Split training data into training and test

In [ ]:
idx = np.random.rand(len(df)) < 0.8
test_df = df[~idx]
train_df = df[idx]
test_df.reset_index(inplace=True)
train_df.reset_index(inplace=True)

In [ ]:
train_df.shape, test_df.shape

In [ ]:
def display_seismic_data(img_num, df):

    #Create a pyplot with two images
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 8))

    #Read Seismic image and corresponding mask
    seismic_img = tf.keras.preprocessing.image.load_img('images/' + df.loc[img_num, 'id'] + '.png', color_mode='grayscale')

    mask_img = tf.keras.preprocessing.image.load_img('masks/' + df.loc[img_num, 'id'] + '.png', color_mode='grayscale')
    
    #Show both images
    ax1.set_title('Seismic')
    ax1.imshow(seismic_img, cmap = 'seismic', interpolation = 'bilinear')
    ax1.axis('off')
    ax2.set_title('Salt')
    ax2.imshow(mask_img, cmap = 'gray', interpolation = 'bilinear')
    ax2.axis('off')
    plt.show()

In [ ]:
#Try random images
img_num = np.random.randint(0, train_df.shape[0])
display_seismic_data(img_num, train_df)

#### Build Batch Generator

In [ ]:
img_size = 128
num_classes = 2

In [ ]:
def batch_generator(df, batch_size=32):

    while True:

        #Create random indexes
        idx = np.random.randint(0, df.shape[0], batch_size)

        #Initialize numpy arrays for X and y
        #Input image is size img_size,img_size,1
        X = np.zeros((batch_size, img_size, img_size,1))
        #Mask's size is img_size, img_size, 1
        y = np.zeros((batch_size, img_size, img_size,num_classes))

        #Populate X and y with actual data
        for i in range(len(idx)):

            #Read image
            img = tf.keras.preprocessing.image.load_img('images/' + df.loc[idx[i],'id'] + '.png', color_mode='grayscale',
                                                        target_size=(img_size, img_size))
            img = tf.keras.preprocessing.image.img_to_array(img).astype('uint8')/255.0

            X[i] = img

            #Read mask
            mask_img = tf.keras.preprocessing.image.load_img('masks/' + df.loc[idx[i],'id'] + '.png',
                                                            color_mode = 'grayscale',
                                                            target_size=(img_size, img_size))
            mask_img = tf.keras.preprocessing.image.img_to_array(mask_img).astype('uint8')/255.0
            mask_img = tf.keras.utils.to_categorical(mask_img, num_classes=num_classes)

            y[i] = mask_img

        yield X, y

In [ ]:
a = batch_generator(df, batch_size=2)

In [ ]:
x, y = next(a)

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
np.unique(y[0])

#### Build Model

Function to create two Convolutional layer block

In [ ]:
def conv2d_block(input_tensor, n_filters):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = tf.keras.layers.Conv2D(n_filters, (3,3), kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # second layer
    x = tf.keras.layers.Conv2D(n_filters, (3,3), kernel_initializer = 'he_normal', padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    return x

Function to build UNET Architecture

In [ ]:
def build_unet(input_img, n_filters=16, dropout=0.1):

    #ENCODER - DOWNSAMPLE the image - 128x128x1

    #First Block
    c1 = conv2d_block(input_img, n_filters*1) #128x128x16
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1) #64x64x16
    p1 = tf.keras.layers.Dropout(dropout)(p1) #64x64x16
    #output will be 64x64x16 for image size 128x128x1

    #Second Block
    c2 = conv2d_block(p1, n_filters*2) #64x64x32
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2) #32x32x32
    p2 = tf.keras.layers.Dropout(dropout)(p2)
    #output will be 32x32x32

    #Third Block
    c3 = conv2d_block(p2, n_filters*4) 
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    p3 = tf.keras.layers.Dropout(dropout)(p3)
    #output will be 16x16x64

    #Fourth Block
    c4 = conv2d_block(p3, n_filters*8) #16x16x128
    p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)
    p4 = tf.keras.layers.Dropout(dropout)(p4)
    ##output will be 8x8x128 

    #Fifth Block
    c5 = conv2d_block(p4, n_filters*16)
    #output will be 8x8x256

    #We now have output of Encoder

    #DECODER - UPSAMPLE the feature to generate mask

    #First Block - connected to fourth block on DOWNSAMPLE side
    u6 = tf.keras.layers.Conv2DTranspose(n_filters * 8, (3, 3), 
                                         strides = (2, 2), 
                                         padding = 'same')(c5) #16x16x128
    u6 = tf.keras.layers.concatenate([u6, c4]) #16x16x256
    u6 = tf.keras.layers.Dropout(dropout)(u6) 
    c6 = conv2d_block(u6, n_filters * 8) #16x16x128

    #Second Block - connected to third block on DOWNSAMPLE side
    u7 = tf.keras.layers.Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6) #32x32x64
    u7 = tf.keras.layers.concatenate([u7, c3])
    u7 = tf.keras.layers.Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4)

    #Third Block - connected to second block on DOWNSAMPLE side
    u8 = tf.keras.layers.Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7) #64x64
    u8 = tf.keras.layers.concatenate([u8, c2])
    u8 = tf.keras.layers.Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2) 

    #Fourth Block - connected to first block on DOWNSAMPLE side
    u9 = tf.keras.layers.Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8) #128x128x16
    u9 = tf.keras.layers.concatenate([u9, c1])
    u9 = tf.keras.layers.Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1) #128 x 128 x 16

    #Build the Output layer
    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(c9) #128x128x1

    #Build the model using different layers
    model = tf.keras.Model(inputs=[input_img], outputs=[outputs])
    return model

Build UNET model

In [ ]:
#Clear out notebook session
tf.keras.backend.clear_session()

#Define input layer
input_img = tf.keras.layers.Input((img_size, img_size, 1), name='input_img')

#Build model
model = build_unet(input_img, dropout=.3)

#Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Build training and test batch generators

In [ ]:
train_generator = batch_generator(train_df,batch_size=32)
test_generator = batch_generator(test_df, batch_size=32)

Train the model

In [ ]:
model.fit(train_generator,
          steps_per_epoch=train_df.shape[0]//32, 
          validation_data=test_generator, 
          validation_steps=test_df.shape[0]//32, 
          epochs=50)

In [ ]:
model = tf.keras.models.load_model('/gdrive/My Drive/TGS.h5')

In [ ]:
model.summary()

#### Model Prediction

In [ ]:
def display_model_prediction(img_num, df):

    #Read Seismic image and corresponding mask
    seismic_img = tf.keras.preprocessing.image.load_img('images/' + df.loc[img_num, 'id'] + '.png', color_mode='grayscale')
    mask_img = tf.keras.preprocessing.image.load_img('masks/' + df.loc[img_num, 'id'] + '.png', color_mode='grayscale')

    #Model prediction
    test_img = seismic_img.resize((img_size, img_size))
    test_img = tf.keras.preprocessing.image.img_to_array(test_img).astype('uint8')/255.0
    test_img = np.expand_dims(test_img, axis=0) #1,128,128,1

    pred = model.predict(test_img) #1,128,128,num_classes
    
    predicted_classes = np.argmax(pred[0], axis=-1)
    #pred[0][pred[0] <0.5] = 0
    #pred[0][pred[0] >=0.5] = 1    
    #print(np.unique(pred[0]))
    #Create a pyplot with two images
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (12, 8))

    #Show both images
    ax1.set_title('Seismic')
    ax1.imshow(seismic_img, cmap = 'seismic', interpolation = 'bilinear')
    ax2.set_title('Actual Salt')
    ax2.imshow(mask_img, cmap = 'gray', interpolation = 'bilinear')
    ax3.set_title('Predicted Salt')
    ax3.imshow(np.reshape(predicted_classes,(img_size, img_size)), cmap = 'gray', interpolation = 'bilinear')


    plt.show()

In [ ]:
img_num = np.random.randint(0, test_df.shape[0])
display_model_prediction(img_num, test_df)

In [ ]:
img_num